# Estimativa de carga aérea

In [2]:
import pandas as pd
import glob

# Testando com microdados_anac

In [3]:
# Ler todos os arquivos e concatená-los em um único DataFrame
microdados_anac = pd.concat(
    [pd.read_csv(arquivo, sep=';', encoding='latin-1', low_memory=False) for arquivo in glob.glob('data/txt/basica*.txt')],
    ignore_index=True  # Para evitar duplicação de índices
)

In [4]:
for i, coluna in enumerate(microdados_anac.columns, start=1):
    print(f"{i}: {coluna}")

1: id_basica
2: id_empresa
3: sg_empresa_icao
4: sg_empresa_iata
5: nm_empresa
6: nm_pais
7: ds_tipo_empresa
8: nr_voo
9: nr_singular
10: id_di
11: cd_di
12: ds_di
13: ds_grupo_di
14: dt_referencia
15: nr_ano_referencia
16: nr_semestre_referencia
17: nm_semestre_referencia
18: nr_trimestre_referencia
19: nm_trimestre_referencia
20: nr_mes_referencia
21: nm_mes_referencia
22: nr_semana_referencia
23: nm_dia_semana_referencia
24: nr_dia_referencia
25: nr_ano_mes_referencia
26: id_tipo_linha
27: cd_tipo_linha
28: ds_tipo_linha
29: ds_natureza_tipo_linha
30: ds_servico_tipo_linha
31: ds_natureza_etapa
32: hr_partida_real
33: dt_partida_real
34: nr_ano_partida_real
35: nr_semestre_partida_real
36: nm_semestre_partida_real
37: nr_trimestre_partida_real
38: nm_trimestre_partida_real
39: nr_mes_partida_real
40: nm_mes_partida_real
41: nr_semana_partida_real
42: nm_dia_semana_partida_real
43: nr_dia_partida_real
44: nr_ano_mes_partida_real
45: id_aerodromo_origem
46: sg_icao_origem
47: sg_iata_

In [5]:
# Correção para nm_municipio_origem = JACAREACANGA e sg_icao_origem = SBSI
microdados_anac.loc[(microdados_anac['nm_municipio_origem'] == 'JACAREACANGA') & 
                    (microdados_anac['sg_icao_origem'] == 'SBSI'), 'sg_icao_origem'] = 'SWSI'

# Correção para nm_municipio_destino = JACAREACANGA e sg_icao_destino = SBSI
microdados_anac.loc[(microdados_anac['nm_municipio_destino'] == 'JACAREACANGA') & 
                    (microdados_anac['sg_icao_destino'] == 'SBSI'), 'sg_icao_destino'] = 'SWSI'

# Correção para nm_municipio_origem = SINOP e sg_icao_origem = SWSI
microdados_anac.loc[(microdados_anac['nm_municipio_origem'] == 'SINOP') & 
                    (microdados_anac['sg_icao_origem'] == 'SWSI'), 'sg_icao_origem'] = 'SBSI'

# Correção para nm_municipio_destino = SINOP e sg_icao_destino = SWSI
microdados_anac.loc[(microdados_anac['nm_municipio_destino'] == 'SINOP') & 
                    (microdados_anac['sg_icao_destino'] == 'SWSI'), 'sg_icao_destino'] = 'SBSI'

# Correção para sg_icao_origem = SSZW
microdados_anac.loc[microdados_anac['sg_icao_origem'] == 'SSZW', 'sg_icao_origem'] = 'SBPG'

# Correção para sg_icao_destino = SSZW
microdados_anac.loc[microdados_anac['sg_icao_destino'] == 'SSZW', 'sg_icao_destino'] = 'SBPG'

# Correção para sg_icao_origem = SWRD
microdados_anac.loc[microdados_anac['sg_icao_origem'] == 'SWRD', 'sg_icao_origem'] = 'SBRD'

# Correção para sg_icao_destino = SWRD
microdados_anac.loc[microdados_anac['sg_icao_destino'] == 'SWRD', 'sg_icao_destino'] = 'SBRD'

In [6]:
def adicionar_coluna_id_aeroporto_origem(microdados_anac):
    microdados_anac['ds_aeroporto_origem'] = microdados_anac['sg_icao_origem'] + ' - ' + microdados_anac['nm_municipio_origem']
    return microdados_anac

def adicionar_coluna_id_aeroporto_destino(microdados_anac):
    microdados_anac['ds_aeroporto_destino'] = microdados_anac['sg_icao_destino'] + ' - ' + microdados_anac['nm_municipio_destino']
    return microdados_anac

microdados_anac = adicionar_coluna_id_aeroporto_origem(microdados_anac)
microdados_anac = adicionar_coluna_id_aeroporto_destino(microdados_anac)

In [24]:
def filtrar_dados(microdados_anac):
    # Filtrando os dados para voos domésticos, regulares e com passageiros pagos > 0
    microdados_anac_filtrada = microdados_anac[
        (microdados_anac['ds_natureza_etapa'] == 'DOMÉSTICA') & 
        (microdados_anac['ds_servico_tipo_linha'] == 'PASSAGEIRO') & 
        (microdados_anac['ds_grupo_di'] == 'REGULAR') & 
        (microdados_anac['nr_passag_pagos'] > 0) & 
        (microdados_anac['nr_rpk'] > 0)
    ].copy()  # Fazendo uma cópia explícita para evitar o aviso

    # Calculando a taxa de ocupação
    microdados_anac_filtrada.loc[:, 'nr_taxa_ocupacao'] = (
        microdados_anac_filtrada['nr_passag_pagos'] / microdados_anac_filtrada['nr_assentos_ofertados']
    )

    # Calculando o peso ocupado por passageiros (75 kg cada)
    microdados_anac_filtrada.loc[:, 'kg_passag_total'] = (
        ((microdados_anac_filtrada['nr_passag_pagos'] + microdados_anac_filtrada['nr_passag_gratis']) * 75)
    )

    # Calculando o peso ocupado por bagagens
    microdados_anac_filtrada.loc[:, 'kg_bagagem_total'] = (
        microdados_anac_filtrada['kg_bagagem_livre'] + microdados_anac_filtrada['kg_bagagem_excesso']
    )

    # Calculando relação de bagagem por passageiro
    microdados_anac_filtrada.loc[:, 'kg_bagagem_por_passag'] = (
        microdados_anac_filtrada['kg_bagagem_total'] / (microdados_anac_filtrada['nr_passag_pagos'] + microdados_anac_filtrada['nr_passag_gratis'])
    )

    # Calculando o peso ocupado por cargas grátis e correios
    microdados_anac_filtrada.loc[:, 'kg_carga_gratis_correio'] = (
        microdados_anac_filtrada['kg_carga_gratis'] + microdados_anac_filtrada['kg_correio']
    )

    # Calculando o peso disponível para carga
    microdados_anac_filtrada.loc[:, 'kg_peso_disponivel'] = (
        microdados_anac_filtrada['kg_payload'] - (microdados_anac_filtrada['kg_passag_total'] + microdados_anac_filtrada['kg_bagagem_total'] + microdados_anac_filtrada['kg_carga_gratis_correio'])
    )

    # Calculando o uso do peso disponível por carga paga
    microdados_anac_filtrada.loc[:, 'nr_uso_carga_paga'] = (
        microdados_anac_filtrada['kg_carga_paga'] / microdados_anac_filtrada['kg_peso_disponivel']
    )

    # Mantendo as colunas na ordem desejada
    colunas_desejadas = [
        'id_basica', 'nm_empresa', 'dt_referencia', 'nm_dia_semana_referencia', 'dt_partida_real', 
        'sg_icao_origem', 'sg_iata_origem', 'ds_aeroporto_origem', 'sg_uf_origem', 'dt_chegada_real', 'sg_icao_destino', 
        'sg_iata_destino', 'ds_aeroporto_destino', 'sg_uf_destino', 'ds_modelo', 'lt_combustivel', 'km_distancia',
        'nr_assentos_ofertados', 'nr_passag_pagos', 'nr_passag_gratis', 'nr_taxa_ocupacao', 'nr_decolagem', 'nr_horas_voadas', 
        'kg_payload', 'kg_carga_paga', 'kg_passag_total', 'kg_bagagem_total', 'kg_bagagem_por_passag',
        'kg_carga_gratis_correio', 'kg_peso_disponivel', 'nr_uso_carga_paga', 
        'nr_velocidade_media', 'nr_ask', 'nr_rpk', 'nr_atk', 'nr_rtk'
    ]
    
    # Garantindo que todas as colunas desejadas estejam presentes no DataFrame e na ordem correta
    microdados_anac_filtrada = microdados_anac_filtrada[colunas_desejadas]

    # Convertendo as colunas de acordo com o prefixo
    for col in microdados_anac_filtrada.columns:
        if col.startswith(('km', 'nr', 'lt', 'kg')):  # Colunas numéricas
            microdados_anac_filtrada[col] = pd.to_numeric(microdados_anac_filtrada[col], errors='coerce')
        elif col.startswith('dt'):  # Colunas de data
            microdados_anac_filtrada[col] = pd.to_datetime(microdados_anac_filtrada[col], errors='coerce').dt.strftime('%Y-%m-%d')
        else:  # O restante das colunas são texto
            microdados_anac_filtrada[col] = microdados_anac_filtrada[col].astype(str)

    # Retornando o DataFrame final
    return microdados_anac_filtrada

# Usando a função para filtrar os dados
microdados_anac_filtrada = filtrar_dados(microdados_anac)

In [30]:
# Agrupando os dados por 'ds_modelo' e aplicando as agregações desejadas
modelo_descricao = microdados_anac_filtrada.groupby('ds_modelo').agg({
    'nr_assentos_ofertados': 'median',
    'nr_passag_pagos': 'median',
    'nr_passag_gratis': 'median',
    'kg_payload': 'median',
    'nr_taxa_ocupacao': 'mean',
    'kg_bagagem_por_passag': 'mean',
    'kg_carga_paga': 'mean',
    'kg_carga_gratis_correio': 'mean',
    'kg_peso_disponivel': 'mean',
    'nr_uso_carga_paga': 'mean'
}).reset_index()

In [31]:
# Filtrando para exibir apenas a linha desejada
modelo_atr = modelo_descricao[modelo_descricao['ds_modelo'].str.contains('ATR 72-600')].transpose()
modelo_atr

,2
ds_modelo,AEROSPATIALE/ALENIA ATR 72-600/72-212A (600)
nr_assentos_ofertados,70.0
nr_passag_pagos,56.0
nr_passag_gratis,1.0
kg_payload,6809.0
nr_taxa_ocupacao,0.751297
kg_bagagem_por_passag,7.30924
kg_carga_paga,117.659587
kg_carga_gratis_correio,1.328097
kg_peso_disponivel,2272.109789
